In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [41]:
job = provider.retrieve_job("cmyk2vbrmwhg008bs57g")
counts = job.result().get_counts()

counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))
print(counts)

{'000 00 00 00': 602, '000 10 00 00': 93, '000 00 10 00': 62, '000 00 00 10': 48, '000 10 10 10': 34, '000 10 10 00': 22, '000 10 00 10': 14, '100 10 00 00': 10, '100 00 00 00': 9, '100 10 00 10': 8, '000 00 01 00': 8, '000 01 00 00': 6, '100 10 10 00': 6, '000 00 00 01': 5, '100 00 10 00': 5, '001 00 00 00': 5, '001 00 01 00': 5, '001 01 00 01': 5, '100 00 00 10': 4, '100 00 10 10': 2, '000 01 01 01': 2, '000 10 00 01': 2, '001 00 01 10': 2, '001 01 00 00': 2, '010 00 00 00': 2, '010 01 00 00': 2, '010 01 10 01': 2, '000 00 10 10': 2, '010 11 00 00': 2, '100 10 10 10': 1, '100 11 00 00': 1, '000 01 01 00': 1, '101 10 00 00': 1, '000 01 01 11': 1, '110 11 00 00': 1, '000 01 11 00': 1, '000 10 01 10': 1, '000 11 00 00': 1, '000 11 11 10': 1, '000 11 11 11': 1, '001 00 00 10': 1, '001 00 10 00': 1, '001 10 10 10': 1, '001 10 11 10': 1, '001 11 00 00': 1, '001 11 01 11': 1, '010 00 10 00': 1, '010 00 11 00': 1, '000 00 10 01': 1, '010 01 10 00': 1, '010 10 00 11': 1, '010 10 01 00': 1, '0

# my d=3, T=3 result
counts = {'000 00 00 00': 602, '000 10 00 00': 93, '000 00 10 00': 62, '000 00 00 10': 48, '000 10 10 10': 34, '000 10 10 00': 22, '000 10 00 10': 14, '100 10 00 00': 10, '100 00 00 00': 9, '100 10 00 10': 8, '000 00 01 00': 8, '000 01 00 00': 6, '100 10 10 00': 6, '000 00 00 01': 5, '100 00 10 00': 5, '001 00 00 00': 5, '001 00 01 00': 5, '001 01 00 01': 5, '100 00 00 10': 4, '100 00 10 10': 2, '000 01 01 01': 2, '000 10 00 01': 2, '001 00 01 10': 2, '001 01 00 00': 2, '010 00 00 00': 2, '010 01 00 00': 2, '010 01 10 01': 2, '000 00 10 10': 2, '010 11 00 00': 2, '100 10 10 10': 1, '100 11 00 00': 1, '000 01 01 00': 1, '101 10 00 00': 1, '000 01 01 11': 1, '110 11 00 00': 1, '000 01 11 00': 1, '000 10 01 10': 1, '000 11 00 00': 1, '000 11 11 10': 1, '000 11 11 11': 1, '001 00 00 10': 1, '001 00 10 00': 1, '001 10 10 10': 1, '001 10 11 10': 1, '001 11 00 00': 1, '001 11 01 11': 1, '010 00 10 00': 1, '010 00 11 00': 1, '000 00 10 01': 1, '010 01 10 00': 1, '010 10 00 11': 1, '010 10 01 00': 1, '010 10 01 10': 1, '010 10 10 00': 1, '010 10 10 10': 1, '010 10 11 00': 1, '010 11 00 11': 1, '000 00 11 00': 1}

In [33]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit_qec.decoders import UnionFindDecoder

# Code params
d = 3
T = 3

code = RepetitionCodeCircuit(d, T)
UF_decoder = UnionFindDecoder(code)


In [34]:
def reorder_string(input_string):
        # Step 1: Split the string by spaces
        split_string = input_string.split()

        # Step 2: Reverse the order of the characters in the first part
        split_string[0] = split_string[0][::-1]

        # Step 3: Join the modified first part with the rest of the split string
        output_string = ' '.join(split_string)

        return output_string

string = '001 01 00 01'
print("Original string:", string)
print("Reordered string:", reorder_string(string))

Original string: 001 01 00 01
Reordered string: 100 01 00 01


In [37]:
logical_counts_without = {'0':0, '1':0}
logical_counts_with = {'0':0, '1':0}

for string, shots in counts.items():  

    logical, flipped_qubit_dict = UF_decoder.process(string, _return_err_str = True, growth=0.5)    
    logical_counts_without[f"{logical[0]}"] += shots

    reordered_string = reorder_string(string)
    logical, flipped_qubit_dict = UF_decoder.process(reordered_string, _return_err_str = True, growth=0.5)
    logical_counts_with[f"{logical[0]}"] += shots
    
print("Logical counts without reordering the final measurement string:", logical_counts_without)
print("Logical counts with reordering the final measurement string:", logical_counts_with)

Logical counts without reordering the final measurement string: {'0': 991, '1': 9}
Logical counts with reordering the final measurement string: {'0': 957, '1': 43}


In [46]:
logical_counts_with = {'0':0, '1':0}

for string, shots in counts.items():  

    logical = UF_decoder.process(string)    
    logical_counts_with[f"{logical[0]}"] += shots


print("logical counts WITH reordering:", logical_counts_with)

logical counts WITH reordering: {'0': 991, '1': 9}
